In [23]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers



In [25]:
from langchain_pinecone.embeddings import PineconeEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
from pinecone import Pinecone

In [11]:
# !pip install pypdf

In [12]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    
    return documents

In [13]:
extracted_text = load_pdf("D:\Medical-Chat-Bot-using-Llama\data")

In [9]:
#extracted_text

In [49]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_huggingface_embedings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [50]:
embeddings = download_huggingface_embedings()


C:\Users\Dell\AppData\Local\Temp\ipykernel_2624\1412518668.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DEC165E0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DCD43A90>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DD7A05E0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2D99A9730>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2D99A9460>
Unclosed client session
client_session: <a

In [51]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
# result = embeddings.embed_query("Hello, world! How are you?")
# print(result)


In [ ]:
def text_splitter(extracted_text):
    text_split = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
    return text_split.split_documents(extracted_data)


In [15]:
text_chunks = text_splitter(extracted_text)

In [ ]:
len(text_chunks)

4251

In [ ]:
# text_chunks
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# print(PINECONE_API_KEY)

In [78]:

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-bot")


In [71]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [72]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

In [ ]:
vector_store.add_documents(documents=text_chunks, ids=uuids)

In [87]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="jlallas/Meta-Llama-3-8B-Instruct-Q4_K_M-GGUF",
    filename="meta-llama-3-8b-instruct-q4_k_m.gguf",
    cache_dir="./models"
)
print("✅ Downloaded to:", model_path)

✅ Downloaded to: ./models\models--jlallas--Meta-Llama-3-8B-Instruct-Q4_K_M-GGUF\snapshots\e05e01f6dfd1aff3c39778d049aa351b233c7439\meta-llama-3-8b-instruct-q4_k_m.gguf


In [94]:
import torch

print("CUDA available?", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))


CUDA available? False
Number of GPUs: 0


In [100]:
from llama_cpp import Llama

llm = Llama(
    model_path=model_path,
    n_ctx=512,
    n_threads=8,
    temperature=0.3,        # Default creativity level
    top_p=0.9,              # Nucleus sampling threshold
    frequency_penalty=0.0,  # Reduce repeated words
    presence_penalty=0.0,   # Penalize repeating existing tokens
    repeat_penalty=1.1,     # Discourage excessive looping
    n_predict=128, 
    use_mmap=True
)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [101]:
response = llm.create_completion(
    prompt="You are a medical assistant. What are the primary symptoms of hypertension?",
    max_tokens=100,
    stop=["\n"]
)

print("🗣️ Response:", response["choices"][0]["text"])

🗣️ Response:  Hypertension, or high blood pressure, is a condition in which the blood pressure in the arteries is elevated above normal levels. The primary symptoms of hypertension may vary depending on the individual and the severity of the condition.


In [102]:
query = "Whate are allergies"
results = vector_store.similarity_search(query, k=2)
print(results)

[Document(metadata={'page': 127.0, 'source': 'D:\\Medical-Chat-Bot-using-Llama\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2114\nAllergies\nGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 114'), Document(metadata={'page': 135.0, 'source': 'D:\\Medical-Chat-Bot-using-Llama\\data\\Medical_book.pdf'}, page_content='foreign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of\nreactions is triggered by harmless, everyday substances.\nThis is the condition known as allergy, and the offend-\ning substance is called an allergen. Common inhaled\nallergens include pollen, dust, and insect parts from tiny\nhouse mites. Common food allergens include nuts, fish,\nand milk.\nAllergic reactions involve a special set of cells in\nthe immune system known as mast cells. Mast cells\nserve as guards in the tissues where the body meets the\noutside world: the skin, the mu

In [113]:
from langchain_community.llms import LlamaCpp

langchain_llm = LlamaCpp(
    model_path=model_path,
    n_ctx=512,
    n_threads=8,
    temperature=0.3,
    top_p=0.9,
    repeat_penalty=1.1,
    n_gpu_layers=0,  # Set >0 if GPU acceleration
    use_mmap=True,
    verbose=True
)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [115]:
prompt_template = """
Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [116]:
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": prompt}


In [117]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=langchain_llm,  # ✅ use the wrapped version
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": 2, "score_threshold": 0.4}
    ),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
while True:
    user_input = input("Enter your question: ")
    result = qa.invoke({"query": user_input})
    print(result["result"])

Acne is a skin condition that occurs when pores or hair follicles become blocked. This allows a waxy material, sebum, to collect inside the pores or follicles. Normally, sebum flows out onto the skin and hair to form a protective coating, but when it cannot get out, small swellings develop on the skin surface. Bacteria and dead skin cells can also collect that can cause inflammation and lead to acne. 224
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 24
Acne is a skin condition that occurs when pores or hair follicles become blocked. This allows a waxy material, sebum, to collect inside the pores or follicles. Normally, sebum flows out onto the skin and hair to form a protective coating, but when it cannot get out, small swellings develop on the skin surface. Bacteria and dead skin cells can also collect that can cause inflammation and lead to acne. 224
GEM - 0001 to 0432 - A  10/22/03 1:41 PM


Llama.generate: prefix-match hit


Acne is not curable, although long-term control is achieved in up to 60% of patients treated with conventional therapies. Wholistic physicians or nutritionists can recommend the proper amounts of herbs such as cnidium seed (Cnidium monnieri) and honeysuckle flower (Lonicera japonica). 
GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26
GALE ENCYCLOPEDIA OF MEDICINE 224
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 24
Question: how can we cure acne 
Answer: Acne is not curable, although long-term control is achieved in up to 60% of patients treated with conventional therapies. Wholistic physicians or nutritionists can recommend the proper amounts of herbs such as cnidium seed (Cnidium monnieri) and honeysuckle flower (Lonicera japonica). 
GALE ENCYCLOPEDIA OF MEDICINE 226
Acne

